In [1]:
import os

In [2]:
%pwd

'/home/isllab/Desktop/ml-end-end/chest-cancer-detection-end-to-end-mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/isllab/Desktop/ml-end-end/chest-cancer-detection-end-to-end-mlflow'

In [61]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sanjoymollarpur/chest-cancer-detection-end-to-end-mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sanjoymollarpur"
os.environ["MLFLOW_TRACKING_PASSWORD"]="8030f3bc3f62e76900ae582df3bb6047a371d1a4"

In [62]:
# MLFLOW_TRACKING_URI=https://dagshub.com/sanjoymollarpur/chest-cancer-detection-end-to-end-mlflow.mlflow \
# MLFLOW_TRACKING_USERNAME=sanjoymollarpur \
# MLFLOW_TRACKING_PASSWORD=8030f3bc3f62e76900ae582df3bb6047a371d1a4 \
# python script.py

In [63]:
import tensorflow as tf

In [64]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [65]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [66]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [67]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/sanjoymollarpur/chest-cancer-detection-end-to-end-mlflow.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [68]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [69]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    # def log_into_mlflow(self):
    #     mlflow.set_registry_uri(self.config.mlflow_uri)
    #     tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
    #     with mlflow.start_run():
    #         mlflow.log_params(self.config.all_params)
    #         mlflow.log_metrics(
    #             {"loss": self.score[0], "accuracy": self.score[1]}
    #         )
    #         # Model registry does not work with file store
    #         if tracking_url_type_store != "file":

    #             # Register the model
    #             # There are other ways to use the Model Registry, which depends on the use case,
    #             # please refer to the doc for more information:
    #             # https://mlflow.org/docs/latest/model-registry.html#api-workflow
    #             mlflow.keras.log_model(self.model, "artifacts/training/model.h5")
    #         else:
    #             mlflow.keras.log_model(self.model, "artifacts/training/model.h5")

            

        # with mlflow.start_run():
        #     mlflow.log_params(self.config.all_params)
        #     mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
        #     mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            print(f"Tracking URI scheme: {tracking_url_type_store}")
    
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                print("Registering the model in Model Registry.")
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                print("Logging the model in a file store.")
                mlflow.keras.log_model(self.model, "model")


In [70]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-01-06 19:17:18,983: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-06 19:17:18,987: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-06 19:17:18,988: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.


2024-01-06 19:17:19.409957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 31s 4s/step - loss: 26.8296 - accuracy: 0.4314
[2024-01-06 19:17:50,530: INFO: common: json file saved at: scores.json]


2024/01/06 19:17:51 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Tracking URI scheme: https
Registering the model in Model Registry.
[2024-01-06 19:17:53,050: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /tmp/tmpkh_un592/model/data/model/assets
[2024-01-06 19:17:53,546: INFO: builder_impl: Assets written to: /tmp/tmpkh_un592/model/data/model/assets]


Successfully registered model 'VGG16Model'.
2024/01/06 19:18:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
